In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45579
2,Huelva,Confirmados PDIA 14 días,275
3,Huelva,Tasa PDIA 14 días,"53,58847945125397"
4,Huelva,Confirmados PDIA 7 días,151
5,Huelva,Tasa PDIA 7 dias,"29,424946898688543"
6,Huelva,Total Confirmados,45781
7,Huelva,Curados,44619
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45579.0


/tmp/ipykernel_3031/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12567.0


/tmp/ipykernel_3031/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3031/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3031/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3031/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 7991 personas en los últimos 7 días 

Un positivo PCR cada 3345 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45579.0,151.0,275.0,513170.0,29.424947,53.588479,38.0
Huelva-Costa,26977.0,81.0,155.0,289548.0,27.974636,53.531711,21.0
Condado-Campiña,14157.0,52.0,92.0,156231.0,33.284047,58.887161,16.0
Cartaya,2013.0,33.0,46.0,20083.0,164.318080,229.049445,7.0
Almonte,2300.0,12.0,18.0,24507.0,48.965602,73.448402,5.0
Moguer,1973.0,11.0,22.0,21867.0,50.304111,100.608222,4.0
Huelva (capital),12567.0,18.0,43.0,143837.0,12.514165,29.894951,4.0
Punta Umbría,1270.0,6.0,7.0,15355.0,39.075220,45.587756,3.0
Palos de la Frontera,1037.0,7.0,10.0,11742.0,59.615057,85.164367,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,80.0,6.0,9.0,1697.0,353.565115,530.347672,0.0
Villarrasa,225.0,3.0,6.0,2211.0,135.685210,271.370421,0.0
Cartaya,2013.0,33.0,46.0,20083.0,164.318080,229.049445,7.0
Manzanilla,141.0,3.0,3.0,2118.0,141.643059,141.643059,2.0
Valverde del Camino,849.0,11.0,15.0,12750.0,86.274510,117.647059,1.0
Moguer,1973.0,11.0,22.0,21867.0,50.304111,100.608222,4.0
Ayamonte,2087.0,11.0,20.0,21104.0,52.122820,94.768764,2.0
Gibraleón,947.0,5.0,12.0,12737.0,39.255712,94.213708,0.0
Paterna del Campo,210.0,2.0,3.0,3457.0,57.853630,86.780445,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,3131.0,4.0,14.0,21393.0,18.697705,65.441967,2.0,0.285714
Aljaraque,1630.0,2.0,6.0,21474.0,9.313589,27.940766,0.0,0.333333
Rociana del Condado,668.0,1.0,3.0,7939.0,12.596045,37.788135,1.0,0.333333
Lepe,3024.0,7.0,19.0,27880.0,25.107604,68.149211,3.0,0.368421
Gibraleón,947.0,5.0,12.0,12737.0,39.255712,94.213708,0.0,0.416667
Huelva (capital),12567.0,18.0,43.0,143837.0,12.514165,29.894951,4.0,0.418605
Moguer,1973.0,11.0,22.0,21867.0,50.304111,100.608222,4.0,0.500000
Villarrasa,225.0,3.0,6.0,2211.0,135.685210,271.370421,0.0,0.500000
Huelva-Costa,26977.0,81.0,155.0,289548.0,27.974636,53.531711,21.0,0.522581
